In [1]:
pip install pyspark requests streamlit


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 83.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 114.6 MB/s eta 0:00:00


In [3]:
"""
train_and_batch_predict.py

- Trains a PySpark ML pipeline on sample (or your) labeled headlines.
- Evaluates on test split (accuracy, precision, recall, f1).
- Fetches a batch of headlines from NewsData (or any API) and predicts.
- Saves the fitted PipelineModel for later streaming/inference.
"""

import os
import requests
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.ml import Pipeline
from pyspark.ml.feature import Tokenizer, HashingTF, IDF, StringIndexer, IndexToString
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import PipelineModel

# ---- Configuration ----
API_KEY = os.environ.get("NEWS_API_KEY", "pub_a09c97e2a49e401c985ff9c287794f90")  # replace with your key or set env var
NEWS_API_ENDPOINT = f"https://newsdata.io/api/1/news?apikey={API_KEY}&language=en"
MODEL_SAVE_PATH = "./saved_pipeline_model"

# ---- Spark session ----
spark = SparkSession.builder \
    .appName("NewsSentimentTrain") \
    .getOrCreate()

# -------- Example training data ----------
# Ideally replace this small example with a real labeled dataset (CSV/Parquet)
training_data = [
    ("Stocks rally after economic boost", "Positive"),
    ("Disaster strikes coastal town", "Negative"),
    ("Tech companies report record profits", "Positive"),
    ("Market crashes amid fears", "Negative"),
    ("Local team wins championship after extra time", "Positive"),
    ("New evidence suggests policy failure", "Negative"),
    ("Breakthrough in renewable energy announced", "Positive"),
    ("Severe storms cause travel disruption", "Negative"),
    ("Company reports higher-than-expected earnings", "Positive"),
    ("Scandal leads to resignation of leader", "Negative")
]

train_df = spark.createDataFrame(training_data, ["headline", "label"])
train_df.show(truncate=False)

# ---- Build pipeline ----
tokenizer = Tokenizer(inputCol="headline", outputCol="words")
hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=2000)
idf = IDF(inputCol="rawFeatures", outputCol="features")
label_indexer = StringIndexer(inputCol="label", outputCol="labelIndex", handleInvalid="keep")
lr = LogisticRegression(featuresCol="features", labelCol="labelIndex", maxIter=20)

# We do NOT include IndexToString in training pipeline because IndexToString needs fitted labels mapping.
pipeline = Pipeline(stages=[tokenizer, hashingTF, idf, label_indexer, lr])

# ---- Train/test split ----
train, test = train_df.randomSplit([0.8, 0.2], seed=42)
fitted_pipeline = pipeline.fit(train)

# ---- Evaluate ----
pred_test = fitted_pipeline.transform(test)

# Multi-class evaluator supports 'accuracy' and 'f1'. For precision/recall you can use the "weightedPrecision"/"weightedRecall"
evaluator_acc = MulticlassClassificationEvaluator(labelCol="labelIndex", predictionCol="prediction", metricName="accuracy")
evaluator_f1 = MulticlassClassificationEvaluator(labelCol="labelIndex", predictionCol="prediction", metricName="f1")
evaluator_precision = MulticlassClassificationEvaluator(labelCol="labelIndex", predictionCol="prediction", metricName="weightedPrecision")
evaluator_recall = MulticlassClassificationEvaluator(labelCol="labelIndex", predictionCol="prediction", metricName="weightedRecall")

acc = evaluator_acc.evaluate(pred_test)
f1 = evaluator_f1.evaluate(pred_test)
prec = evaluator_precision.evaluate(pred_test)
rec = evaluator_recall.evaluate(pred_test)

print(f"Evaluation on test set -> Accuracy: {acc:.4f}, F1: {f1:.4f}, Precision: {prec:.4f}, Recall: {rec:.4f}")

# ---- Convert numeric prediction back to original labels properly ----
# After fitting, find the label order from the fitted StringIndexer stage
# In our pipeline, stage index 3 is label_indexer (0:tokenizer,1:hashingTF,2:idf,3:label_indexer,4:lr)
fitted_label_indexer = fitted_pipeline.stages[3]
labels = fitted_label_indexer.labels  # list like ['Positive','Negative'] in some order
print("Label indexer labels (index -> label):")
for i, lbl in enumerate(labels):
    print(f"  {i} -> {lbl}")

index_to_string = IndexToString(inputCol="prediction", outputCol="Predicted_Label", labels=labels)

# Apply converter to predictions and show
predictions_with_labels = index_to_string.transform(pred_test)
predictions_with_labels.select("headline", "label", "Predicted_Label", "probability").show(truncate=False)

# ---- Save the fitted pipeline model (so we can load and use in streaming / dashboard) ----
# We will save the fitted pipeline WITHOUT IndexToString - we'll save it separately as a small convenience file.
# Save the fitted pipeline (it contains the tokenizer, tf, idf, label_indexer and lr)
fitted_pipeline.write().overwrite().save(MODEL_SAVE_PATH)
print("Saved fitted pipeline to:", MODEL_SAVE_PATH)

# Save the labels mapping to a tiny file for IndexToString usage later
# We'll write a plain text file with labels
labels_path = os.path.join(MODEL_SAVE_PATH, "labels.txt")
with open(labels_path, "w", encoding="utf-8") as f:
    f.write("\n".join(labels))
print("Saved labels to:", labels_path)

# ---- Batch fetch headlines and predict (demo) ----
def fetch_headlines(limit=10):
    try:
        resp = requests.get(NEWS_API_ENDPOINT, timeout=15)
        resp.raise_for_status()
        data = resp.json()
        results = data.get("results", []) if isinstance(data, dict) else []
        titles = [r.get("title") for r in results if r.get("title")]
        return titles[:limit]
    except Exception as e:
        print("Warning: fetch failed:", e)
        # return demo headlines on failure
        return [
            "Stocks rally after economic boost",
            "Unexpected market turbulence causes losses",
            "Local startup secures major funding"
        ]

batch_headlines = fetch_headlines(limit=20)
print("Fetched headlines (batch):", batch_headlines[:5])

# Load saved pipeline and predict
loaded_pipeline = PipelineModel.load(MODEL_SAVE_PATH)

# Create spark DF and predict
predict_df = spark.createDataFrame([(h,) for h in batch_headlines], ["headline"])
preds = loaded_pipeline.transform(predict_df)

# apply IndexToString with labels we saved
# Read saved labels
with open(labels_path, "r", encoding="utf-8") as f:
    saved_labels = [l.strip() for l in f if l.strip()]

index_to_string = IndexToString(inputCol="prediction", outputCol="Predicted_Label", labels=saved_labels)
preds = index_to_string.transform(preds)
preds.select("headline", "Predicted_Label", "probability").show(truncate=False)

# Done
spark.stop()


+---------------------------------------------+--------+
|headline                                     |label   |
+---------------------------------------------+--------+
|Stocks rally after economic boost            |Positive|
|Disaster strikes coastal town                |Negative|
|Tech companies report record profits         |Positive|
|Market crashes amid fears                    |Negative|
|Local team wins championship after extra time|Positive|
|New evidence suggests policy failure         |Negative|
|Breakthrough in renewable energy announced   |Positive|
|Severe storms cause travel disruption        |Negative|
|Company reports higher-than-expected earnings|Positive|
|Scandal leads to resignation of leader       |Negative|
+---------------------------------------------+--------+

Evaluation on test set -> Accuracy: 0.5000, F1: 0.3333, Precision: 0.2500, Recall: 0.5000
Label indexer labels (index -> label):
  0 -> Negative
  1 -> Positive
+---------------------------------------